In [251]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
#Traitement des messages d'erreur :
warnings.filterwarnings('ignore')
import timeit
#Librairie pour gérer l'espace mémoire du PC
import gc
#Librairie pour créer un dataset de train et un de test
from sklearn.model_selection import train_test_split
#Librairie pour l'encodage des variables catégorielles
from sklearn.preprocessing import LabelEncoder
#Librairies pour réaliser un pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
# Importation du package pour le SMOTE
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import make_pipeline
import math
#Librairie pour importer des variables d'un autre notebook
import pickle
#Librairie pour les métriques avec GridSearch
import sklearn.metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report, confusion_matrix
#Librairie pour XGBoostClassifier
from xgboost import XGBClassifier

### Fonctions utilisées dans le notebook

In [252]:
#Création d'une fonction pour utiliser chaque modèle d'entrainement et afficher les différents résultats de métriques

def training_model (name_of_model, parameters):
    """Fonction afin d'entrainer différents modèle en retournant les résultats de différents métriques"""
    grid = GridSearchCV(name_of_model,
                       parameters,
                       #Cross validation
                       cv=5, #Permet d'entrainer le modèle en le découpant en folds, ici on entrainte 5 fois le modèle 
                       scoring=accuracy_score,
                       n_jobs=-1, #Permet d'obliger le modèle à travailler sur tous les CPU du PC
                       refit='accuracy_score', #Permet de réajuster un estimateur en utilisant les meilleurs paramètres trouvés sur l'ensemble de données
                       return_train_score=True, #Permet de renvoyer les scores de train
                       verbose=1) #Permet de définir la quantité d'infos renvoyer par GridSearch pendant son déroulement
                                             
    model = make_pipeline(StandardScaler(with_mean=False), grid)
    
    start_time = timeit.default_timer()
    model.fit(X_train_res, y_train_res)
    elapsed = timeit.default_timer() - start_time
    elapsed = np.round(elapsed, 3)

    print("Le temps d'entrainement du modèle est de {0:.3f} millisecondes".format(elapsed), "\n")

   # "precision": "accuracy_score", "f1": "f1_score", "recall": "recall_score",
   #                             "roc_auc": "roc_auc_score"

    #Affichage des meilleurs paramètres
    best_params = grid.best_params_
    
    #Affichage des meilleurs métriques :
    best_estimators = grid.best_estimator_
    #best_scores = grid.best_estimator_.score(X_test_transformed, y_test),
    
    print("Les meilleurs paramètres du modèle sont : ", best_params)
    #Affichage des résultats
    class_report = classification_report(y_test, model.predict(X_test_transformed))
    print(class_report)
    confusion_mat(y_test, model.predict(X_test_transformed))
    AUC_graph(y_test, model.predict(X_test_transformed), model.predict_proba(X_test_transformed)[:,1])
    
    
    accuracy = accuracy_score(y_test, model.predict(X_test_transformed))
    print("Le score accuracy du modèle est de : ", accuracy)
    #auc_score = roc_auc_score(y_test, model.predict(X_test_transformed))
    #print("Le score AUC du modèle est de : ", auc_score)
    f1 = f1_score(y_test, model.predict(X_test_transformed))
    print("Le score f1 du modèle pour la classe 1 est de : ", f1)
    precision = precision_score(y_test, model.predict(X_test_transformed))
    print("Le score precision du modèle pour la classe 1 est de : ", precision)
    recall = recall_score(y_test, model.predict(X_test_transformed))
    print("Le score recall du modèle pour la classe 1 est de : ", recall)

In [253]:
#Fonction pour affiche la matrice de confusion
def confusion_mat(y_true, y_pred):
    """Fonction pour calculer et afficher une matrice de conusion"""
    conf_mat = confusion_matrix(y_true, y_pred)
    fig = plt.figure(figsize=(10,10))
    group_names = ['Vrais Négatifs', 'Faux Positifs', 'Faux Négatifs', 'Vrais Positifs']
    group_counts = ["{0:0.0f}".format(value) for value in conf_mat.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in conf_mat.flatten()/np.sum(conf_mat)]
    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)
    sns.heatmap(conf_mat, annot=labels, fmt="", cmap='Blues')
    
#Fonction pour afficher la courbe AUC
def AUC_graph(y_true, y_pred, y_pred_proba):
    """Fonction pour calculer et afficher la courbe AUC """
    dummy_class_report = classification_report(y_true, y_pred)
    model_roc_auc_score = roc_auc_score(y_test, y_pred_proba)
    fig = plt.figure(figsize=(10,10))
    false_positive, true_positive, _ = roc_curve(y_true, y_pred_proba)
    plt.plot(false_positive, true_positive, label = "AUC = %0.2f" %model_roc_auc_score)
    plt.plot([0, 1], [0, 1],linestyle='--')
    plt.xlabel("Taux de Faux Positifs")
    plt.ylabel("Taux de Vrais Positifs")
    plt.title("Courbe de l'AUC")
    plt.legend(loc = 'upper left')
    plt.show()
    return model_roc_auc_score, dummy_class_report

In [254]:
#Fonction pour préparer le pipeline qui traitera les variables numériques
def Preprocessing (numeric):
    """Fonction pour préparer le pipeline afin de pouvoir traiter les variables numériques"""
    #On prend la médiane concernant le traitement des Nan et on standardise les données à l'aide de RobustScaler
    #Ce Scaler supprime la médiane et met à l'échelle les données en fonction de l'intervalle des quantiles
    numeric_transfs = [('imputer',SimpleImputer(missing_values= np.NAN, strategy= 'median')),('scaler', RobustScaler())]
    numeric_pipeline = Pipeline(numeric_transfs)
    all_transfs = [("numeric",numeric_pipeline,numeric)]
    full_preprocessor = ColumnTransformer(all_transfs, remainder='passthrough')
    return full_preprocessor

#Fonction utilisée pour calculer la taille du dataset avant et après avoir réalisé le SMOTE
def resampling (features, target, resample):
    """Fonction utilisée pour calculer la taille du dataset avant et après avoir réalisé le SMOTE"""
    print('Original dataset shape %s' % Counter(target))
    X, y = resample.fit_resample(features, target)
    print('Resampled dataset shape %s' % Counter(y))
    return X, y

### Lecture du dataset

In [255]:
path = 'Projet_7/'
data_final = pd.read_csv(path + "data_final.csv")

In [256]:
#Affichage de la totalité de la ligne 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth',3000)
display(data_final.head())

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,OWN_CAR_AGE,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_CITY_NOT_WORK_CITY,TOTALAREA_MODE,DEF_30_CNT_SOCIAL_CIRCLE,FLAG_DOCUMENT_3,AMT_REQ_CREDIT_BUREAU_YEAR,CREDIT_ACTIVE,DAYS_CREDIT,AMT_CREDIT_SUM,AMT_BALANCE,AMT_PAYMENT_CURRENT,SK_DPD_x,SK_DPD_DEF,DAYS_INSTALMENT_delay,AMT_INSTALMENT_delta,AMT_ANNUITY,AMT_CREDIT_y,AMT_DOWN_PAYMENT,DAYS_DECISION,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_LAST_DUE,DAYS_TERMINATION,NAME_CONTRACT_STATUS,CODE_REJECT_REASON,NAME_CLIENT_TYPE,CNT_INSTALMENT_FUTURE,SK_DPD_y
0,100002,1,Cash loans,M,0,202500.0,406597.5,Unaccompanied,working,low_level_education,not_married,0.018801,-9461,-637,NaN,other,1.0,2,2,10,0,0.0149,2.0,1,1.0,Closed,-874.00,108131.945625,NaN,NaN,NaN,NaN,-20.421053,0.0,9251.775,179055.00,0.00,-606.000000,24.000000,365243.0,-25.000000,-17.000000,Approved,XAP,New,15.000000,0.0
1,100003,0,Cash loans,F,0,270000.0,1293502.5,Family,working,high_level_education,married,0.003541,-16765,-1188,NaN,other,2.0,1,1,11,0,0.0714,0.0,1,0.0,Closed,-1400.75,254350.125000,NaN,NaN,NaN,NaN,-7.000000,0.0,56553.990,484191.00,3442.50,-1305.000000,10.000000,365243.0,-1054.333333,-1047.333333,Approved,XAP,Refreshed,4.909091,0.0
2,100004,0,Revolving loans,M,0,67500.0,135000.0,Unaccompanied,working,low_level_education,not_married,0.010032,-19046,-225,26.0,other,1.0,2,2,9,0,NaN,0.0,0,0.0,Closed,-867.00,94518.900000,NaN,NaN,NaN,NaN,-3.000000,0.0,5357.250,20106.00,4860.00,-815.000000,4.000000,365243.0,-724.000000,-714.000000,Approved,XAP,New,0.000000,0.0
3,100006,0,Cash loans,F,0,135000.0,312682.5,Unaccompanied,working,low_level_education,married,0.008019,-19005,-3039,NaN,other,2.0,2,2,17,0,NaN,0.0,1,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,-19.375000,0.0,23651.175,291695.50,34840.17,-272.444444,23.000000,365243.0,182477.500000,182481.750000,Approved,XAP,Repeater,8.650000,0.0
4,100007,0,Cash loans,M,0,121500.0,513000.0,Unaccompanied,working,low_level_education,not_married,0.028663,-19932,-3038,NaN,other,1.0,2,2,11,1,NaN,0.0,0,0.0,Closed,-1149.00,146250.000000,NaN,NaN,NaN,NaN,-6.320000,0.0,12278.805,166638.75,3390.75,-1222.833333,20.666667,365243.0,72136.200000,72143.800000,Approved,XAP,Repeater,11.666667,0.0


### Sélection des features

Nous allons décrire chacune des variables et stipuler si nous les conservons ou non.
Cela limitera aussi la quantité de données affichées ultérieurement sur notre dashboard pour une meilleure lisibilité.

- SK_ID_CURR : Identifiant du prêt --> Variable conservée
- TARGET : 0 client sans incident et 1 client avec des difficultés de remboursement --> Variable conservée
- NAME_CONTRACT_TYPE : Type de prêt contracté --> Variable conservée
- CODE_GENDER : Sexe de l'emprunteur --> Variable supprimée pour éviter tout sexisme dans le scoring
- CNT_CHILDREN : Nombre d'enfant de l'emprunteur --> Variable conservée (impact sur la capacité de remboursement)
- AMT_INCOME_TOTAl : Montant total des revenus --> Variable conservée (impact sur la capacité de remboursement)
- AMT_CREDIT_x : Montant total des crédits --> Variable supprimée au profit de la variable plus utile AMT_CREDIT_SUM
- NAME_TYPE_SUITE : Indication concernant l'accompagnement de notre client --> Variable supprimée car non utile pour l'octroi d'un prêt (utile en cas de conquête de nouveaux clients)
- NAME_INCOME_TYPE = Provenance des revenus percus par le client --> Variable supprimée car non utile pour l'octroi d'un prêt (utile lors de la collecte d'épargne)
- NAME_EDUCATION_TYPE : Information surle type d'éducation du client (pré ou post BAC) --> Variable supprimée car peut être considerée comme de le descrimination dans le cadre de l'octroi d'un prêt (respect du volet RGPD)
- NAME_FAMILY_STATUS : Situation familiale du client --> Variable conservée (Impact sur le nombre d'emprunteur et rentre dans le volet KYC)
- REGION_POPULATION_RELATIVE : Information sur la situtation géographique du client --> Variable non conservée au profit de la variable plus utile REGION_RATING_CLIENT
- DAYS_BIRTH : Date d'anniversaire du client --> Variable conservée car impacte l'ADE adossée au prêt (et rentre dans le cadre du KYC)
- DAYS_EMPLOYED : Nombre de jours d'emploi du client --> Variable supprimée pour éviter tout sexisme (les femmes en congés maternité pourraient avoir plus de difficultés à emprunter)
- OWN_CAR_AGE : Information sur la possession de sa proprevoirture --> Variable non conservée car non utile pour l'octroi d'un crédit
- OCCUPATION_TYPE : Type d'habitation du client --> Variable supprimée car non utile pour l'octroi d'un prêt
- CNT_FAM_MEMBERS : Nombre de membre de la famille --> Variable supprimée au profit des variable sur le nombre d'enfant et du régime matrimoniale qui donnent déjà les informations nécessaires
- REGION_RATING_CLIENT : Evaluation de la région où habite le client --> Variable conservée (le lieu d'habitation peut avoir un impact sur la fidelité et la connaissance du client)
- REGION_RATING_CLIENT_W_CITY : Evaluation de la région où vit le client en tenant compte de la ville --> Variable supprimée au profit de la variable plus utile REGION_RATING_CLIENT
- HOUR_APPR_PROCESS_START : Heure où le client a demandé le prêt --> Variable supprimée car non utile pour l'octroi d'un prêt
- REG_CITY_NOT_WORK_CITY : Clients qui n'habitent pas dans la ville où ils travaillent --> Variable supprimée au profit de la variable plus utile REGION_RATING_CLIENT
- TOTALAREA_MODE : Information sur la taille des habitations des clients --> Variable supprimée car non utile pour l'octroi d'un prêt
- DEF_30_CNT_SOCIAL_CIRCLE : Observations sociales sur les clients --> Variable supprimée car non utile pour l'octroi d'un prêt (Cette information prise en compte pourrait être considérée comme de la descrimination)
- FLAG_DOCUMENT_3 : Document à fournir à la banque --> Variable supprimée car non utile pour l'octroi d'un crédit (ce document est nécessaire pour toute étude et donc sera obligatoire en cas d'acceptation mais aussi de refus lors du montage du dossier)
- AMT_REQ_CREDIT_BUREAU_YEAR : Nombre de demandes de renseignements au bureau de crédit concernant le client un jour par an --> Variable conservée car elle permet de juger du relationnel avec la banque (un contact régulier augmente la fidelité du client)
- CREDIT_ACTIVE : Statut des crédits déclarés par le bureau de crédit --> Variable supprimée au profit de la variable plus utile AMT_CREDIT_SUM
- DAYS_CREDIT : Nombre de jours depuis la dernière demande de crédit réalisée par le client --> Variable supprimée car non utile vue la volatilité des taux d'emprunts et les changements de situation pro et perso des clients
- AMT_CREDIT_SUM : Montant total des prêts --> Variable conservée
- AMT_BALANCE : Montant des soldes clients --> Varaible supprimée car non utile lors de l'octroi d'un crédit
- AMT_PAYMENT_CURRENT : Montants des paiments courants --> Variable supprimée car non utilisée pour calculer le taux d'entement d'un prêt
- SK_DPD_x : jours de retard au cours du mois du crédit précédent --> Variable supprimée car un retard d'une seule échéance ne justifie pas du refus de l'octroi d'un crédit (situation exceptionnelle peut arriver à tout moment en revanche si nous avions le nombre de retard de paiement cela aurait été plus pertinent)
- SK_DPD_DEF : Jours de retard au cours d'une période donnée du crédit précédent --> Variable supprimée car un retard d'une seule échéance ne justifie pas du refus de l'octroi d'un crédit (situation exceptionnelle peut arriver à tout moment en revanche si nous avions le nombre de retard de paiement cela aurait été plus pertinent) 
- DAYS_INSTALMENT_delay : le temps entre la date du commencement des paiments et la date du début des échelonnements du crédit --> Variable conservée
- AMT_INSTALMENT_delta : La différence entre les montants des paiemants de crédits et les montants des échelonnements --> Variable conservée
- AMT_ANNUITY : Montant des annuités --> Variable supprimée car nous savons déjà quels sont les montants des crédits
- AMT_CREDIT_y : Montant du crédit de la demande précédente --> Variable supprimée au profit de la variable AMT_CREDIT_SUM 
- AMT_DOWN_PAYMENT : Acompte sur la demande précédente --> Variable supprimée car non utile pour l'octroi d'un crédit
- DAYS_DECISION : Nombre de jours pour la validation de la dernière offre de prêt via l'application --> Variable supprimée car non utile pour l'octroi d'un crédit
- CNT_PAYMENT : Durée du crédit précédent souscrit pour la demand de crédit --> Variable supprimée car non utile pour l'octroi d'un crédit
- DAYS_FIRST_DRAWING : Date du premier déblocage du précédent prêt via l'application --> Variable supprimée car non utile pour l'octroi d'un crédit
- DAYS_LAST_DUE : Derniere date de la demande de prêt --> Variable supprimée car non utile pour l'octroi d'un crédit
- DAYS_TERMINATION : Date de fin de la demande précédente --> Variable supprimée car non utile pour l'octroi d'un crédit
- NAME_CONTRACT_STATUS : État du contrat au cours du mois --> Variable supprimée car nous somme au moment de l'octroi du crédit et cette variable donne une information sur un prêt déjà accepté et en cours ou non de déblocage
- CODE_REJECT_REASON : Motif de rejet de la précédente demande de prêt --> Variable supprimée pour éviter tout à priori sur la demande de prêt
- NAME_CLIENT_TYPE : Information sur le statut du client (client ou non client) lors de la dernière demande de prêt --> Variable supprimée car non utile lors de l'octroi d'un crédit (elle pourra être nécessaire pour évaluer le taux de fidelité du client)
- CNT_INSTALMENT_FUTURE : Acomptes restant à payer sur le crédit précédent --> Variable supprimée au profit de la variable AMT_CREDIT plus utile.
- SK_DPD_y : Jours de retard au cours du mois du crédit précédent --> Variable supprimée car un retard d'une seule échéance ne justifie pas du refus de l'octroi d'un crédit (situation exceptionnelle peut arriver à tout moment en revanche si nous avions le nombre de retard de paiement cela aurait été plus pertinent)

In [257]:
#Création du dataset avec les variables conservées :
data_work = data_final.copy()
del data_final
list_to_delete = ["CODE_GENDER", "AMT_CREDIT_x", "NAME_TYPE_SUITE", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", 
                 "REGION_POPULATION_RELATIVE", "DAYS_EMPLOYED", "OWN_CAR_AGE", "OCCUPATION_TYPE", "CNT_FAM_MEMBERS",
                 "REGION_RATING_CLIENT_W_CITY", "HOUR_APPR_PROCESS_START", "REG_CITY_NOT_WORK_CITY",
                 "TOTALAREA_MODE", "FLAG_DOCUMENT_3", "CREDIT_ACTIVE", "DAYS_CREDIT", "AMT_BALANCE",
                 "AMT_PAYMENT_CURRENT", "SK_DPD_x", "SK_DPD_DEF", "AMT_ANNUITY", "AMT_CREDIT_y", "AMT_DOWN_PAYMENT",
                 "DAYS_DECISION", "CNT_PAYMENT", "DAYS_FIRST_DRAWING", "DAYS_LAST_DUE", "DAYS_TERMINATION",
                 "NAME_CONTRACT_STATUS", "CODE_REJECT_REASON", "NAME_CLIENT_TYPE", "CNT_INSTALMENT_FUTURE", "SK_DPD_y",
                 "DEF_30_CNT_SOCIAL_CIRCLE"]
data_work.drop(list_to_delete, axis =1, inplace = True)
display(data_work.head())
print(data_work.shape)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_FAMILY_STATUS,DAYS_BIRTH,REGION_RATING_CLIENT,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_CREDIT_SUM,DAYS_INSTALMENT_delay,AMT_INSTALMENT_delta
0,100002,1,Cash loans,0,202500.0,not_married,-9461,2,1.0,108131.945625,-20.421053,0.0
1,100003,0,Cash loans,0,270000.0,married,-16765,1,0.0,254350.125000,-7.000000,0.0
2,100004,0,Revolving loans,0,67500.0,not_married,-19046,2,0.0,94518.900000,-3.000000,0.0
3,100006,0,Cash loans,0,135000.0,married,-19005,2,NaN,NaN,-19.375000,0.0
4,100007,0,Cash loans,0,121500.0,not_married,-19932,2,0.0,146250.000000,-6.320000,0.0


(307511, 12)


In [258]:
#On conserve la colonne des identifiants dans un dataset différent pour pas que cela impact le modèle
data_id = data_work["SK_ID_CURR"]
#On sépare la variable recherchée TARGET dans un autre dataset
data_target = data_work["TARGET"]
#On supprime ensuite ces deux variables du dataset original
data_work.drop(["SK_ID_CURR", "TARGET"], axis = 1, inplace = True)
display(data_id.head())
print(data_id.shape)
display(data_target.head())
print(data_target.shape)
display(data_work.head())
print(data_work.shape)
#Export des fichiers data_id et data_target en CSV
pd.DataFrame(data_target).to_csv(path + 'data_target.csv',index=False)
pd.DataFrame(data_id).to_csv(path + 'data_id.csv',index=False)

0    100002
1    100003
2    100004
3    100006
4    100007
Name: SK_ID_CURR, dtype: int64

(307511,)


0    1
1    0
2    0
3    0
4    0
Name: TARGET, dtype: int64

(307511,)


,NAME_CONTRACT_TYPE,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_FAMILY_STATUS,DAYS_BIRTH,REGION_RATING_CLIENT,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_CREDIT_SUM,DAYS_INSTALMENT_delay,AMT_INSTALMENT_delta
0,Cash loans,0,202500.0,not_married,-9461,2,1.0,108131.945625,-20.421053,0.0
1,Cash loans,0,270000.0,married,-16765,1,0.0,254350.125000,-7.000000,0.0
2,Revolving loans,0,67500.0,not_married,-19046,2,0.0,94518.900000,-3.000000,0.0
3,Cash loans,0,135000.0,married,-19005,2,NaN,NaN,-19.375000,0.0
4,Cash loans,0,121500.0,not_married,-19932,2,0.0,146250.000000,-6.320000,0.0


(307511, 10)


Nous avons donc x_train qui est représenté par data_work et y_train par data_target.
Nous allons maintenant créer nos datasets de train et de test et les préparer pour utiliser notre modèle

In [259]:
data_work.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   NAME_CONTRACT_TYPE          307511 non-null  object 
 1   CNT_CHILDREN                307511 non-null  int64  
 2   AMT_INCOME_TOTAL            307511 non-null  float64
 3   NAME_FAMILY_STATUS          307511 non-null  object 
 4   DAYS_BIRTH                  307511 non-null  int64  
 5   REGION_RATING_CLIENT        307511 non-null  int64  
 6   AMT_REQ_CREDIT_BUREAU_YEAR  265992 non-null  float64
 7   AMT_CREDIT_SUM              263490 non-null  float64
 8   DAYS_INSTALMENT_delay       251957 non-null  float64
 9   AMT_INSTALMENT_delta        251957 non-null  float64
dtypes: float64(5), int64(3), object(2)
memory usage: 23.5+ MB


In [260]:
#On renomme les colonnes pour une meilleure compréhension
data_work.rename(columns = {"NAME_CONTRACT_TYPE": "Type_de_pret", "CNT_CHILDREN": "Nombre_d_enfants",
                            "AMT_INCOME_TOTAL": "Montant_des_revenus", "NAME_FAMILY_STATUS" : "Regime_matrimonial",
                            "DAYS_BIRTH" : "Age", "REGION_RATING_CLIENT" : "Note_region_client",
                            "AMT_REQ_CREDIT_BUREAU_YEAR" : "Nombre_de_demande_client", "AMT_CREDIT_SUM" : "Montant_des_credits",
                            "DAYS_INSTALMENT_delay": "Delai_anticipation_pret", "AMT_INSTALMENT_delta": "Montant_anticipation_pret"}, inplace = True)
display(data_work.head())

,Type_de_pret,Nombre_d_enfants,Montant_des_revenus,Regime_matrimonial,Age,Note_region_client,Nombre_de_demande_client,Montant_des_credits,Delai_anticipation_pret,Montant_anticipation_pret
0,Cash loans,0,202500.0,not_married,-9461,2,1.0,108131.945625,-20.421053,0.0
1,Cash loans,0,270000.0,married,-16765,1,0.0,254350.125000,-7.000000,0.0
2,Revolving loans,0,67500.0,not_married,-19046,2,0.0,94518.900000,-3.000000,0.0
3,Cash loans,0,135000.0,married,-19005,2,NaN,NaN,-19.375000,0.0
4,Cash loans,0,121500.0,not_married,-19932,2,0.0,146250.000000,-6.320000,0.0


In [261]:
#Remplacement des valeurs des colonnes pour une meilleure compréhension
print(data_work['Type_de_pret'].unique())
data_work['Type_de_pret'] = data_work['Type_de_pret'].replace('Cash loans', 'Prêts de trésorerie')
data_work['Type_de_pret'] = data_work['Type_de_pret'].replace('Revolving loans', 'Prêts renouvelables')
print(data_work['Nombre_d_enfants'].unique())
#Nous avons jusqu'à 19 enfants dans notre étude
#On enlève les chiffres après la virgule pour les montant des revenus
data_work['Montant_des_revenus'] = data_work['Montant_des_revenus'].astype(np.int)
print(data_work['Regime_matrimonial'].unique())
data_work['Regime_matrimonial'] = data_work['Regime_matrimonial'].replace('not_married', 'célibataire')
data_work['Regime_matrimonial'] = data_work['Regime_matrimonial'].replace('married', 'marié(e)')
data_work['Regime_matrimonial'] = data_work['Regime_matrimonial'].replace('other', 'autre')
#Traitement de la date de naissance en repassant la valeur en positif
data_work['Age'] = ((data_work['Age'])/-360).astype(np.int)
print(data_work['Note_region_client'].unique())
#Nous avons 3 type de région avec le 3 comme meilleur région et le 1 la moins bonne
print(data_work['Nombre_de_demande_client'].unique())
#Nous avons de 0 à 25 demandes réalisées par le client
#On enlève les chiffres après la virgule pour les montant des crédits
print(data_work['Montant_des_credits'].max())
#data_work['Montant_des_credits'] = data_work['Montant_des_credits'].astype(np.int)
#On enlève les chiffres après la virgule pour les délais d'anticipation et on rend le résultat positif
print(data_work['Delai_anticipation_pret'].min())
print(data_work['Delai_anticipation_pret'].max())
#data_work['Delai_anticipation_pret'] = (data_work['Delai_anticipation_pret']*-1).astype(np.int)
#On enlève les chiffres après la virgule pour les montants d'anticipation
print(data_work['Montant_anticipation_pret'].min())
print(data_work['Montant_anticipation_pret'].max())
#data_work['Montant_anticipation_pret'] = (data_work['Montant_anticipation_pret']).astype(np.int)
display(data_work.head())

['Cash loans' 'Revolving loans']
[ 0  1  2  3  4  7  5  6  8  9 11 12 10 19 14]
['not_married' 'married' 'other']
[2 1 3]
[ 1.  0. nan  2.  4.  5.  3.  8.  6.  9.  7. 10. 11. 13. 16. 12. 25. 23.
 15. 14. 22. 17. 19. 18. 21. 20.]
198072344.25
-392.0
605.0
-169033.935
555000.0


,Type_de_pret,Nombre_d_enfants,Montant_des_revenus,Regime_matrimonial,Age,Note_region_client,Nombre_de_demande_client,Montant_des_credits,Delai_anticipation_pret,Montant_anticipation_pret
0,Prêts de trésorerie,0,202500,célibataire,26,2,1.0,108131.945625,-20.421053,0.0
1,Prêts de trésorerie,0,270000,marié(e),46,1,0.0,254350.125000,-7.000000,0.0
2,Prêts renouvelables,0,67500,célibataire,52,2,0.0,94518.900000,-3.000000,0.0
3,Prêts de trésorerie,0,135000,marié(e),52,2,NaN,NaN,-19.375000,0.0
4,Prêts de trésorerie,0,121500,célibataire,55,2,0.0,146250.000000,-6.320000,0.0


In [264]:
#Réorganisation des colonnes
data_work = data_work[['Type_de_pret', 'Age', 'Regime_matrimonial', 'Nombre_d_enfants', 'Montant_des_revenus',
                       'Montant_des_credits', 'Note_region_client', 'Nombre_de_demande_client',
                       'Delai_anticipation_pret', 'Montant_anticipation_pret']]
display(data_work.head())

,Type_de_pret,Age,Regime_matrimonial,Nombre_d_enfants,Montant_des_revenus,Montant_des_credits,Note_region_client,Nombre_de_demande_client,Delai_anticipation_pret,Montant_anticipation_pret
0,Prêts de trésorerie,26,célibataire,0,202500,108131.945625,2,1.0,-20.421053,0.0
1,Prêts de trésorerie,46,marié(e),0,270000,254350.125000,1,0.0,-7.000000,0.0
2,Prêts renouvelables,52,célibataire,0,67500,94518.900000,2,0.0,-3.000000,0.0
3,Prêts de trésorerie,52,marié(e),0,135000,NaN,2,NaN,-19.375000,0.0
4,Prêts de trésorerie,55,célibataire,0,121500,146250.000000,2,0.0,-6.320000,0.0


In [265]:
#Export du dataset data_work en CSV
pd.DataFrame(data_work).to_csv(path + 'data_work.csv',index=False)

Ce notebook a du être découpé en deux parties suite à des incidents techniques lors du lancement
Fin de la première partie du notebook P7_Part6_Features_And_Final_Model_Part1.